In [41]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
%reload_ext autoreload

In [43]:
# logging 
import logging
import logging.config

# allow loading modules from strings
import importlib

# parse configuration files
import configparser   

# shell utilities for manpiulating files
from shutil import copyfile, copyfileobj

# parse command line arguments
import argparse

# handle file paths in a sane way
from pathlib import Path

# clock 
from datetime import datetime

# loop delay - sleep
import time

# handle kill/interrupt signals gracefully
import signal

# https requests - downloading album art, querying lms server
import requests

######### PyPi libraries
# LMS Query interface
import lmsquery

# create a cache path to hold downloade album art
from cachepath import CachePath, TempPath, Path

# rate limit the queries on the LMS server
from ratelimiter import RateLimiter

In [44]:
# local libraries
import epdlib

In [45]:
def arg_parse():
    '''parse arguments for this script
    
    Arguments parsed:
        -c: configuration file
    
    Returns:
        :obj:`argpars.ArgumentParser`'''
    parser = argparse.ArgumentParser()
    
    # configuration file
    parser.add_argument('-c', type=str, required=False,
                        help='use the specified configuration file. Default is stored in ~/.config/myApp/config.ini')
    
    #FIXME add log level with -v -vv -vvv etc.
    
    parser.add_argument('-q', type=str, required=False,
                       help='this is for testing only')
    
    args, unknown = parser.parse_known_args()
    logging.info(f'discarding unknwon commandline arguments: {unknown}')
    
    return args

In [46]:
def read_config(cfgfile=None, default=None):
    '''read `cfgfile` file and optionally create one if it does not exist

    Args:
        cfgfile(str): path to configuration file to be read
        default(str): path to default configuration file that should be 
            used if cfgfile does not exist -- this is useful for creating
            config files on first run
    
    Returns:
        obj:configparser.ConfigParser
    '''
 
    if not cfgfile:
        return {}
    
    cfgfile = Path(cfgfile).expanduser().resolve()
    default = Path(default).expanduser().resolve() if default else None
    
    # check if the path exists
    logging.debug(f'creating parent directory (if needed): {cfgfile.parent}')
    Path(cfgfile.parent).mkdir(parents=True, exist_ok=True)

    # check if specified file exists, otherwise copy the default (if provided)    
    if not cfgfile.exists() and default:
        logging.debug(f'copying {default} to {cfgfile}')
        try:
            copyfile(default, cfgfile)
        except (FileNotFoundError, PermissionError) as e:
            print(f'failed to copy default file ({default}) to specified file: {e}')
            return False
#     elif not default:
#         raise FileNotFoundError(f'could not open {cfgfile}')
    
    config = configparser.ConfigParser()
    logging.info(f'reading config file {cfgfile}')
    config.read(cfgfile)
    
    return config

In [47]:
class cache_art():
    '''download and cache album artwork from lms server
    
    Args:
        app_name(`str`): name of application - used to create /tmp/`app_name` directory
        
    Properties:
        app_name(`str`): name of application
        cache_path(:obj:`CachePath`): path to store cached artwork
        '''
    def __init__(self, app_name):
        self.app_name = app_name
        
    @property
    def app_name(self):
        '''name of application
            this is used to set the path in /tmp/`app_name`
            
        Sets:
            app_name(`str`): name of application
            cache_path(obj:`CachePath`): path to store cached artwork'''
        return self._app_name
    
    @app_name.setter
    def app_name(self, app_name):
        self._app_name = app_name
        self.cache_path = CachePath(app_name, dir=True)
    
    def cache_artwork(self, artwork_url=None, album_id=None):
        '''download and cache artwork as needed from lms server
        
        Args:
            artwork_url(`str`): URL of artwork on LMS server
            album_id(`str`): unique string that identifies artwork on LMS server
            
        Returns:
            (obj:`pathlib.Path`): path to album artwork'''
        if not artwork_url or not album_id:
            raise TypeError(f'missing required value: artwork_url: {artwork_url}, album_id: {album_id}')
                
        album_id = str(album_id)
        
        artwork_path = self.cache_path/(album_id+'.jpg')
        
        if artwork_path.exists():
            logging.debug(f'artwork previously cached')
            return artwork_path
        
        r = False
        try:
            r = requests.get(artwork_url, stream=True)
        except requests.exceptions.RequestException as e:
            logging.error(f'failed to fetch artwork at: {artwork_url}: {e}')
        
        if r:
            try:
                with open(artwork_path, 'wb') as outFile:
                    copyfileobj(r.raw, outFile)
                    logging.debug(f'wrote ablum artwork to: {artwork_path}')
            except (OSError, FileExistsError, ValueError) as e:
                logging.error(f'failed to write {artwork_path}')
        else:
            logging.error('failed to download album art due to previous errors')
            return None
        
        return artwork_path

    def clear_cache(self, force=False):
        '''clear the contents of the cache folder or wipe entirely
        
        Args:
            force(`bool`): False (default) - delete the files, True - delete entire directory and contents
            '''
        logging.debug(f'clearing previously downloaded files in {self.cache_path}')
        if force:
            logging.info(f'removing cache directory: {self.cache_path}')
            try:
                self.cache_path.rm()
            except Exception as e:
                logging.error(f'Error removing cach directory: {e}')
                return False
        else:
            try:
                self.cache_path.clear()
            except Exception as e:
                logging.error(f'Error removing files in cache d directory: {e}')
                return False
            
        return True

In [48]:
def config_2dict(configuration):
    '''convert an argparse object into a dictionary
    
    Args:
        configuration(:obj:`argpars.ArgumentParser`)
        
    Returns:
        `dict`'''
    d = {}
    for section in configuration.sections():
        d[section] = {}
        for opt in configuration.options(section):
            d[section][opt] = configuration.get(section, opt)
    
    return d

In [49]:
def time_now():
    '''return current time in HH:MM format
        e.g. 04:44, 23:16
    Returns:
        `str`'''
    return datetime.now().strftime("%H:%M")

In [50]:
def limit_callback(until):
    '''callback function for RateLimiter; provides logging information'''
    duration = int(round(until - time.time()))
    logging.debug(f'function call rate limited sleeping for {duration} seconds')

In [51]:
class signalHandler(object):
    '''handle specific signals and allow graceful exiting while loop
        https://stackoverflow.com/a/31464349/5530152
    
    Signals Handled Gracefully:
        SIGINT
        SIGTERM
    Atributes:
        kill_now (bool) default: False
    '''
    kill_now = False
    def __init__(self):
        signal.signal(signal.SIGINT, self.exit_gracefully)
        signal.signal(signal.SIGTERM, self.exit_gracefully)
    
    def exit_gracefully(self, signum, frame):
        self.kill_now = True

In [52]:
# class configuration():
#     '''configuration for main loop
    
#     Attributes:
#         app_name(`str): name of application
#         developer(`str`): domain of developer in reverse DNS format e.g. com.txoof
#         app_long_name(`str`): dotted concat of developer, app_name'''
    
#     def __init__(self, app_name=None, developer=None, builtin_cfg=None, user_cfg=None,
#                  lms_server_cfg={}):
#         self.developer = developer
#         self.app_name = app_name
#         self.builtin_cfg = builtin_cfg
#         self.user_cfg = user_cfg
#         self.lms_server_cfg = lms_server_cfg
        
#     @property
#     def app_name(self):
#         '''application name
    
#         Sets:
#             app_name(`str`): short application name (e.g. slimpi)
#             app_long_name(`str`): dotted concat of developer, app_name'''        
#         return self._app_name
    
#     @app_name.setter
#     def app_name(self, app_name):
#         self._app_name = app_name
#         if self.developer:
#             self.app_long_name = '.'.join([self.developer, self.app_name])
            
#     @property
#     def builtin_cfg(self):
#         return self._builtin_cfg
    
#     @builtin_cfg.setter
#     def builtin_cfg(self, builtin_cfg):
#         '''configuration file distributed with application
    
#         Sets:
#             builtin_cfg(:obj:`pathlib.Path`): path to distribution configuration file
#             user_cfg(:obj:`pathlib.Path`): path to user configuration file'''
#         if builtin_cfg:
#             self._builtin_cfg = Path(builtin_cfg).resolve()
#             self.user_cfg = Path('~/.config').expanduser().resolve()/self.app_long_name/self.builtin_cfg.name
#         else:
#             self._builtin_cfg = None
            
#     @property
#     def user_cfg(self):
#         return self._user_cfg
    
#     @user_cfg.setter
#     def user_cfg(self, user_cfg):
#         if user_cfg:
#             self._user_cfg = user_cfg
    
#     @property
#     def lms_server_cfg(self):
#         '''set the lms server configuration and init an lmsquery object
        
#         Sets:
#             lms_server_cfg(`dict`): {"host": IP/Hostname, "port": 9000, "player_id": hex:id }'''
#         return self._lms_server_cfg
    
#     @lms_server_cfg.setter
#     def lms_server_cfg(self, lms_server_cfg):
#         self._lms_server_cfg = lms_server_cfg
        
    
            
    
    

In [67]:
def main():
    
    import constants
    # execute the main loop (set to false to stop main loop execution)
    execute = True

    ########### constants
    # waveshare library
    waveshare = constants.waveshare
    # version number
    version = constants.version
    # image for albums that fail to return valid artwork
    noartwork = Path(constants.noartwork).resolve()
    # layouts file 
    layouts_file = constants.layouts
    # long name - reverse dotted dns of developer, app name
    appLongName = '.'.join([constants.devel_name, constants.app_name])
    
    # setup logging
    logingConfig = Path('./logging.cfg').resolve()
    logging.config.fileConfig(logingConfig)
    logger = logging.getLogger(__name__)
    
    # Define signal handler for killing process
    sigHandler = signalHandler()
    
    ########## Configuration Variables
    # Default configuration file if none is specified
    builtin_cfg = Path('./slimpi.cfg').resolve()
    # standard location for stored user configuration
    user_cfg = Path(f'~/.config/{appLongName}/slimpi.cfg').expanduser()  

    
    # parse command line arguments
    args = arg_parse()
    
    # use a specified configuration file
    if args.c:
        user_cfg = args.c
        
    logging.info(f'using configuration file: {user_cfg}')        
    
    # read configuration file
    # config parser object
    configuration = read_config(cfgfile=user_cfg, default=builtin_cfg)
    execute = True if configuration else False
    # create a dictionary from the config parser object
    cfg = config_2dict(configuration)
    
    # get all the configuration variables 
    try:
        # lms server settings
        lms_server = cfg['lms_server']
        
        # set epd type - Raises KeyError
        epd = importlib.import_module('.'.join([waveshare, cfg['layouts']['display']]))
        
        # set layout types - Raises AttirbuteError, KeyError
        layouts = importlib.import_module(layouts_file)
        playing_layout_format = getattr(layouts, cfg['layouts']['now_playing'])
        stopped_layout_format = getattr(layouts, cfg['layouts']['stopped'])
        splash_layout_format = getattr(layouts, cfg['layouts']['splash'])
        
        show_splash = cfg['main']['splash_screen']

        # set log level - Raises KeyError
        log_level = cfg['logging']['log_level']
        logger.setLevel(log_level)
        
    except (KeyError) as e:
        logging.error(f'error locating section/option in configuration file: {e}')
        execute = False
    except AttributeError as e:
        logging.error(f'error accessing attribute: {e}')
        execute = False

    #################### setup environment ####################

    
    ########## LMS Query object
    # rate limiting class - allow a maximum of `max_calls` in per `period`
    lmsQuery_ratelimit = RateLimiter(max_calls=1, period=3) # callback=limit_callback
    
    try:
        if lms_server['host'] and lms_server['port']:
            logging.info(f'explicitly setting LMS server, port and player id: {lms_server}')
            lms = lmsquery.LMSQuery(host=lms_server['host'], port=lms_server['port'], 
                                    player_id=lms_server['player_id'])
        else:
            logging.info(f'host and port not set in configuration file; setting only player_id')
            lms = lmsquery.LMSQuery(player_id=lms_server['player_id'])
            
    except KeyError as e:
        #FIXME add humanfriendly option to find server and specify player id
        logging.warning(f'missing configuration value for lms_server: {e}')
        execute = False
        
    
    ########## Layout objects - for managing display blocks
    resolution = [epd.EPD_HEIGHT, epd.EPD_WIDTH]
    # reverse sort to put longest dimension first (landscape display)
    resolution.sort(reverse=True)    
    
    splash_layout = epdlib.Layout(layout=splash_layout_format, resolution=resolution)
    splash_layout.update_contents({'app_name': constants.app_name, 
                                   'version': constants.version, 
                                   'url': constants.url})
    music_layout = epdlib.Layout(layout=playing_layout_format, resolution=resolution)
    stopped_layout = epdlib.Layout(layout=stopped_layout_format, resolution=resolution)
#     stopped_layout.update_contents({'time': time_now(), 'mode': ' '})
    
    
    ########## Screen objects - for managing writing to screen
    screen = epdlib.Screen()
    screen.epd = epd.EPD()
    screen.initEPD()
#     screen.elements=music_layout.blocks.values()
    # display the splash screen
    if show_splash:
        screen.elements = splash_layout.blocks.values()
        screen.concat()
        screen.writeEPD()
        time.sleep(5)
    
    ########## Flow control variables
    refresh = False # refresh the screen 

    ########## Now-Playing and album art  Values
    nowplaying_id = None
    nowplaying_mode = None
    artwork_cache = cache_art(appLongName)
        
    # if execute was set false anywhere, bail out
    if not execute:
        print('Exiting due to previous errors - see log file for more information')


    try:
        # loop forever, watching for a kill signal
        while not sigHandler.kill_now:
            # clear the response 
            response = None


            
            with lmsQuery_ratelimit:
                try:
                    logging.debug(f'querying lms server for status of player: {lms.player_id}')
                    response = lms.now_playing()
                except requests.exceptions.RequestException.ConnectionError as e:
                    logging.warning(f'server could not find active player with player_id: {lms.player_id}')
                    logging.warning('is the specified player active?')
                    response = {'title': f'Could not connect to meidia player: {player_id}',
                                'id': 'NONE', 'mode': 'ERROR'}
                    nowplaying_mode = response['mode']
                    
                except KeyError as e:
                    logging.info(f'No playlist is active on player_id: {lms.player_id}')
                    response = {'title': 'No music is queued', 'id': 'NONE', 'mode': 'No Playlist'}
                    nowplaying_mode = response['mode']
            
            if response:
                resp_id = response['id']
                resp_mode = response['mode']
                if resp_id != nowplaying_id or resp_mode != nowplaying_mode:
                    logging.info(f'track/mode change {resp_mode}')
                    nowplaying_id = resp_id
                    nowplaying_mode = resp_mode
                    
                    # add the path to the album art into the response
                    # FIXME wrap in a try:
                    artwork = artwork_cache.cache_artwork(response['artwork_url'], response['album_id'])
                    if not artwork:
                        artwork = noartwork
                    response['coverart'] = str(artwork)
                             
                    music_layout.update_contents(response)
                    refresh = music_layout
                else:
                    refresh = False

            if nowplaying_mode != 'play' and screen.update.last_updated > 60:
                logging.debug('music appears to be paused')
                stopped_layout.update_contents({'time': time_now(), 'mode': nowplaying_mode})
                refresh = stopped_layout
                
            # only refresh if needed
            if refresh:
                logging.debug('refreshing display')
                screen.initEPD()
                screen.elements=refresh.blocks.values()
                screen.concat()
                screen.writeEPD()   
                refresh = False        
            
            
            # sleep half the time
            time.sleep(0.5)
    finally:
        logging.info("cleaning up and wiping screen")
        artwork_cache.clear_cache()
        screen.initEPD()
        screen.clearEPD()    
            
    

    
    
#     return cfg

In [ ]:
if __name__ == '__main__':
    foo = main()
print(foo)

<ipython-input-45-611d20285b5a>:arg_parse:21:INFO - discarding unknwon commandline arguments: ['-f', '/home/pi/.local/share/jupyter/runtime/kernel-68101631-ced9-4f41-bbda-3f668c7b102a.json']
<ipython-input-67-b33f4e9cd0f6>:main:41:INFO - using configuration file: /home/pi/.config/com.txoof.slimpi/slimpi.cfg
<ipython-input-46-81fa3b8ea00e>:read_config:21:DEBUG - creating parent directory (if needed): /home/pi/.config/com.txoof.slimpi
<ipython-input-46-81fa3b8ea00e>:read_config:36:INFO - reading config file /home/pi/.config/com.txoof.slimpi/slimpi.cfg
<ipython-input-67-b33f4e9cd0f6>:main:90:INFO - host and port not set in configuration file; setting only player_id
Layout:__init__:82:DEBUG - no font specified yet
Layout:layout:199:DEBUG - calculating values from layout for resolution [600, 448]
Layout:_calculate_layout:232:DEBUG - ***app_name***
Layout:_check_keys:98:DEBUG - checking key/values
Layout:_check_keys:103:DEBUG - missing key: maxchar; adding and setting to None
Layout:_check_k

Layout:_scalefont:155:DEBUG - target Y dimension: 89.6
Layout:_scalefont:173:DEBUG - Y target exceeded
Layout:_scalefont:185:DEBUG - test string: W W W ; dimensions for fontsize 78: (222, 90)
Layout:_calculate_layout:232:DEBUG - ***coverart***
Layout:_check_keys:98:DEBUG - checking key/values
Layout:_check_keys:103:DEBUG - missing key: rand; adding and setting to False
Layout:_check_keys:103:DEBUG - missing key: inverse; adding and setting to False
Layout:_check_keys:103:DEBUG - missing key: maxchar; adding and setting to None
Layout:_check_keys:103:DEBUG - missing key: dimensions; adding and setting to None
Layout:_calculate_layout:239:DEBUG - dimensions: (240, 192)
Layout:_calculate_layout:249:DEBUG - has calculated position
Layout:_calculate_layout:270:DEBUG - abs_coordinates: (0, 0)
Layout:_calculate_layout:232:DEBUG - ***artist***
Layout:_check_keys:98:DEBUG - checking key/values
Layout:_check_keys:103:DEBUG - missing key: rand; adding and setting to False
Layout:_check_keys:103:D

Layout:_calculate_layout:232:DEBUG - ***mode***
Layout:_check_keys:98:DEBUG - checking key/values
Layout:_check_keys:103:DEBUG - missing key: maxchar; adding and setting to None
Layout:_check_keys:103:DEBUG - missing key: dimensions; adding and setting to None
Layout:_calculate_layout:239:DEBUG - dimensions: (600, 56)
Layout:_calculate_layout:249:DEBUG - has calculated position
Layout:_calculate_layout:270:DEBUG - abs_coordinates: (0, 0)
Layout:_scalefont:125:DEBUG - no max char set; using: 6
Layout:_scalefont:140:DEBUG - calculating font size
Layout:_scalefont:141:DEBUG - using text: W W W ; maxchar: 6
Layout:_scalefont:142:DEBUG - using font at path: /home/pi/src/slimpi_epd/fonts/Ubuntu/Ubuntu-Regular.ttf
Layout:_scalefont:154:DEBUG - target X font dimension 510.0
Layout:_scalefont:155:DEBUG - target Y dimension: 39.199999999999996
Layout:_scalefont:173:DEBUG - Y target exceeded
Layout:_scalefont:185:DEBUG - test string: W W W ; dimensions for fontsize 41: (147, 40)
Layout:_set_image

Block:_text2image:395:DEBUG - text image dimensions: (202, 24)
Block:_text2image:432:DEBUG - v-center image coordinates
Block:_text2image:436:DEBUG - image coordinates (240, 372)
Layout:update_contents:340:DEBUG - ignoring block artwork_url
Layout:update_contents:340:DEBUG - ignoring block time
Layout:update_contents:337:DEBUG - updating block: mode
Block:text_formatter:365:DEBUG - formatted list:
 ['play']
Block:_text2image:382:DEBUG - random is set: True
Block:_text2image:395:DEBUG - text image dimensions: (33, 25)
Block:_text2image:416:DEBUG - setting random position within area
Layout:update_contents:337:DEBUG - updating block: coverart
Block:image:178:DEBUG - formatting image: /tmp/com.txoof.slimpi/5697.jpg
Block:image:180:DEBUG - set image dimensions: 182
Block:image:210:DEBUG - set img_coordinates: (30, 261)
<ipython-input-67-b33f4e9cd0f6>:main:189:DEBUG - refreshing display
epd5in83:ReadBusy:69:DEBUG - e-Paper busy
epd5in83:ReadBusy:72:DEBUG - e-Paper busy release
Screen:initEP

Block:_text2image:416:DEBUG - setting random position within area
Layout:update_contents:337:DEBUG - updating block: coverart
Block:image:178:DEBUG - formatting image: /tmp/com.txoof.slimpi/4991.jpg
Block:image:180:DEBUG - set image dimensions: 182
Block:image:210:DEBUG - set img_coordinates: (29, 261)
<ipython-input-67-b33f4e9cd0f6>:main:189:DEBUG - refreshing display
epd5in83:ReadBusy:69:DEBUG - e-Paper busy
epd5in83:ReadBusy:72:DEBUG - e-Paper busy release
Screen:initEPD:167:INFO - <waveshare_epd.epd5in83.EPD object at 0xaf487a70> initialized
Screen:concat:150:DEBUG - pasting image at: (21, 83)
Screen:concat:150:DEBUG - pasting image at: (29, 261)
Screen:concat:150:DEBUG - pasting image at: (240, 286)
Screen:concat:150:DEBUG - pasting image at: (240, 373)
Screen:concat:150:DEBUG - pasting image at: (240, 416)
epd5in83:getbuffer:128:DEBUG - imwidth = 600  imheight =  448 
epd5in83:ReadBusy:69:DEBUG - e-Paper busy
epd5in83:ReadBusy:72:DEBUG - e-Paper busy release
epd5in83:ReadBusy:69:

In [ ]:
import sys
sys.argv.append('-c')
sys.argv.append('./slimpi.cfg')

In [ ]:
sys.argv
